In [ ]:
from postprocessing import *
import healpy as hp
import glob
import os
from natsort import natsorted

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
def momentum_flux(directory, i_file):
    filename = "snap_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    v_square = vx ** 2 + vy ** 2 + vz ** 2
    j = snap_data['PartType0/Density'] * v_square
    return j, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
def mass_flux(directory, i_file, proj='radial'):
    filename = "snap_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    v_r = (vx * (x - 500) + vy * (y - 500) + vz * (z - 500)) / np.sqrt(x ** 2 + y ** 2 + z ** 2)
    if proj == 'radial':
        j = snap_data['PartType0/Density'] * v_r
    elif proj == 'transverse':
        v_t = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2 - v_r ** 2)
        j = snap_data['PartType0/Density'] * v_t
    elif proj == 'total':
        v = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2)
        j = snap_data['PartType0/Density'] * v
    else:
        raise ValueError("Wrong projection type. Expected 'radial', 'transverse', or 'total'. ")
    return j, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
def total_energy(directory, i_file):
    filename = "snap_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    v_square = vx ** 2 + vy ** 2 + vz ** 2
    E = snap_data['PartType0/Masses'] * v_square / 2 + snap_data['PartType0/InternalEnergy']
    return E, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m8/jet42_early')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

## mass flux propogation in r

$j = \rho \cdot v_r $

In [ ]:
i_file = 8
projection = 'radial'
j_all, x_all, y_all, z_all, time = mass_flux(output_directory, i_file, proj=projection)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20))
    j, x, y, z = j_all[mask], x_all[mask], y_all[mask], z_all[mask]
    j_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        j_shell.append(j[distance.argmin()])
    
    if projection == 'radial':
        hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
        min=-5e2, max=5e2, unit="mass flux in cgs", cmap='coolwarm')
    else:
        hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=10, max=5e4, unit="mass flux in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'massflux_mollview_{projection}_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/massflux_mollview_{projection}_{i_file}_*.png'
ofilename = figures_directory + f'/massflux_mollview_{projection}_{i_file}-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## mass flux propogation in t

In [ ]:
radius = 200
projection = 'radial'
j_all, x_all, y_all, z_all, time = mass_flux(output_directory, i_file, proj=projection)

In [ ]:
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi) * radius

In [ ]:
for i_file in range(12):
    j_all, x_all, y_all, z_all, time = momentum_flux(output_directory, i_file)
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 10) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 10))
    j, x, y, z = j_all[mask], x_all[mask], y_all[mask], z_all[mask]
    j_shell = []

    for vector in vec:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        j_shell.append(j[distance.argmin()])

    hp.mollview(np.array(j_shell), title=fr"Mass flux in $r \in$ [{np.round(radius,0) - 10}; {np.round(radius,0) + 10}] pc", 
            min=200, max=5e5,
            unit="mass flux in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'massflux_mollview_{projection}_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/massflux_mollview_{projection}*{radius}.png'
ofilename = figures_directory + f'/massflux_mollview_{projection}-time-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## momentum flux distribution

In [ ]:
# radius_grid = np.linspace(rho.min(), rho.max(), 101)

In [ ]:
# result = []
# for i in range(len(radius_grid) - 1):
#     radius_i0 = radius_grid[i]
#     radius_i1 = radius_grid[i + 1]
    
#     mask = (rho > radius_i0) & (rho < radius_i1)
#     result.append([(radius_i0 + radius_i1) / 2, j[mask].mean()])

In [ ]:
# result = np.array(result).T

In [ ]:
for i in range(7):
    j, rho, phi, theta, time = momentum_flux(output_directory, i)
#     fig = plt.figure()
    plt.scatter(rho, j, s=1, label=f'{np.round(time, 2)} Myr')
    plt.xlabel('r')
    plt.ylabel('mean momentum flux')
plt.legend(loc='upper left')
plt.xlim(0, 500)

## total energy flux

In [ ]:
i_file = 8
E_all, x_all, y_all, z_all, time = total_energy(output_directory, i_file)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20))
    E, x, y, z = E_all[mask], x_all[mask], y_all[mask], z_all[mask]
    E_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        E_shell.append(E[distance.argmin()])
    
    hp.mollview(np.array(E_shell), title=fr"Total energy in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=1e5, max=1e8,
            unit="total energy in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'totalenergy_mollview_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/totalenergy_mollview_8_*.png'
ofilename = figures_directory + '/totalenergy_mollview_8-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)